# Gov Exploratory Analysis - 1st step

### Etapa de engenharia de dados.
#### Este script realiza a busca dos dados no repositório do datasus para criar o dataset a ser utilizado na etapa de análise de dados. Busca todos os dados disponíveis dos 27 estados brasileiros desde 2006 até o final de 2023. Todos os dados são obtidos em formato prioritário do próprio datasus (.dbc) e é utilizado um script terceiro [(dbc2csv)](https://github.com/greatjapa/dbc2csv) para realizar a conversão para csv.

### Author: Marcos Vinicius Ribeiro Silva

## Params
### Os parâmetros podem ser alterados a fim de se obter um dataset diferente para a análise. Neste estudo de caso foi utilizado os anos da pandemia no Brasil (Do início de 2020 ao final de 2021) em comparação com os 3 anos anteriores (2017, 2018 e 2019)

In [ ]:
pandemic_years = [20, 21] # pandemic years in Brazil
years_compare_before = [17, 18, 19] # years to compare sets in hospital before pandemic years
years_compare_after = [22] # years to compare sets in hospital after pandemic years
download_data = True # param to download data (set True if you don't have the data)

## Libraries

In [ ]:
import urllib.request
import os
import pandas as pd

## Generate link and get all files for generate dataset

In [ ]:
file_name = 'LT{}{}{}'
file_link = 'ftp://ftp.datasus.gov.br/dissemin/publicos/CNES/200508_/Dados/LT/{}.dbc'

In [ ]:
path_data = 'data'
months = [x for x in range(1, 13)] # all months in years
years = [x for x in range(2006, 2023)] # data available in datasus
states = ['AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG',
          'MS', 'MT', 'PA', 'PB',  'PE', 'PI', 'PR', 'RJ', 'RN', 'RO', 'RR',
         'RS', 'SC', 'SE', 'SP', 'TO'] # all stats in Brazil

In [ ]:
if download_data:
    if path_data not in os.listdir(os.getcwd()):
        os.mkdir(path_data)
    
    for year in years:
        for month in months:
            for state in states:
                syear = str(year)[2:4:]
                smonth = str(month).zfill(2)
                download_name = file_name.format(state, syear, smonth)
                download_link = file_link.format(download_name)
                try:
                    out_file = os.path.join(path_data, '{}.dbc'.format(download_name))
                    urllib.request.urlretrieve(download_link, out_file)
                except:
                    pass

## Convert data to .csv

In [ ]:
if download_data:
    path_files = os.path.join(os.getcwd(), path_data)
    
    if 'dbc2csv' not in os.listdir(os.getcwd()):
        !git clone https://github.com/greatjapa/dbc2csv

    !docker build -t dbc2csv dbc2csv/
    !docker run -it -v $path_files:/usr/src/app/data dbc2csv make

## Generate new dataset with .csv files

In [ ]:
csv_path_files = os.path.join(path_data, 'csv')
new_data = pd.DataFrame()
validate_years = pandemic_years + years_compare_before + years_compare_after

if len(validate_years) > 0:
    for file in os.listdir(csv_path_files):
        if '.csv' in file:
            state, year, month = file[2:4], int(file[4:6]), int(file[6:8])
            if year in validate_years:
                print(file)
                aux_data = pd.read_csv(os.path.join(csv_path_files, file), encoding='unicode_escape')
                aux_data['MES'] = month
                aux_data['ANO'] = year
                aux_data['ESTADO'] = state
                new_data = pd.DataFrame.append(new_data, aux_data)

In [ ]:
new_data.head()
new_data.to_csv('data.csv', index=False)

## New dataset generated